In [20]:
#! pip install ydata_profiling==4.7.0

  Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl.metadata (61 kB)


Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [3]:
import numpy as np 
import pandas as pd
import math as m 
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import random

In [4]:
df = pd.read_csv('objets-trouves-restitution.csv',sep=';')

df

,Date,Date et heure de restitution,Gare,Code UIC,Nature d'objets,Type d'objets,Type d'enregistrement
0,2018-08-16T08:46:55+02:00,2018-08-16T10:52:08+02:00,Nantes,87481002.0,Téléphone portable,"Appareils électroniques, informatiques, appare...",Objet trouvé
1,2018-08-16T08:50:34+02:00,NaN,Redon,87471300.0,"Casque (vélo, moto)","Vélos, trottinettes, accessoires 2 roues",Objet trouvé
2,2018-08-16T09:11:18+02:00,NaN,Béziers,87781005.0,"Sac d'enseigne (plastique, papier, …)","Bagagerie: sacs, valises, cartables",Objet trouvé
3,2018-08-16T09:19:47+02:00,NaN,Rennes,87471003.0,"Manteau, veste, blazer, parka, blouson, cape","Vêtements, chaussures",Objet trouvé
4,2018-08-16T09:23:18+02:00,NaN,Paris Montparnasse,87391003.0,Sac à dos,"Bagagerie: sacs, valises, cartables",Objet trouvé
...,...,...,...,...,...,...,...
907271,2024-08-30T19:59:22+02:00,2024-09-04T12:34:26+02:00,Strasbourg,87212027.0,"Valise, sac sur roulettes","Bagagerie: sacs, valises, cartables",Objet trouvé
907272,2024-08-30T20:55:27+02:00,2024-09-05T12:49:08+02:00,Lille Europe,87223263.0,"Valise, sac sur roulettes","Bagagerie: sacs, valises, cartables",Objet trouvé
907273,2024-08-30T23:23:44+02:00,NaN,Nîmes,87775007.0,Sac à dos,"Bagagerie: sacs, valises, cartables",Objet trouvé
907274,2024-08-30T23:36:00+02:00,2024-08-31T12:43:30+02:00,Nice,87756056.0,"Ordinateur, ordinateur portable, notebook prot...","Appareils électroniques, informatiques, appare...",Objet trouvé


## Analyse de la base de données ( à completer) 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907276 entries, 0 to 907275
Data columns (total 7 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Date                          907276 non-null  object 
 1   Date et heure de restitution  350898 non-null  object 
 2   Gare                          906197 non-null  object 
 3   Code UIC                      906197 non-null  float64
 4   Nature d'objets               907276 non-null  object 
 5   Type d'objets                 907276 non-null  object 
 6   Type d'enregistrement         907276 non-null  object 
dtypes: float64(1), object(6)
memory usage: 48.5+ MB


### Analyse de la mémoire vive consommée 

In [5]:
df.memory_usage(index=True).sum()

50807588

### Analyse des doublons

In [1]:
df.nunique().sum()

NameError: name 'df' is not defined

## Valeurs manquantes

### Analyse valeurs manquantes 

In [39]:
df = pd.read_csv('objets-trouves-restitution.csv',sep=';',parse_dates=['Date et heure de restitution'])

df

,Date,Date et heure de restitution,Gare,Code UIC,Nature d'objets,Type d'objets,Type d'enregistrement
0,2018-08-16T08:46:55+02:00,2018-08-16 10:52:08+02:00,Nantes,87481002.0,Téléphone portable,"Appareils électroniques, informatiques, appare...",Objet trouvé
1,2018-08-16T08:50:34+02:00,NaT,Redon,87471300.0,"Casque (vélo, moto)","Vélos, trottinettes, accessoires 2 roues",Objet trouvé
2,2018-08-16T09:11:18+02:00,NaT,Béziers,87781005.0,"Sac d'enseigne (plastique, papier, …)","Bagagerie: sacs, valises, cartables",Objet trouvé
3,2018-08-16T09:19:47+02:00,NaT,Rennes,87471003.0,"Manteau, veste, blazer, parka, blouson, cape","Vêtements, chaussures",Objet trouvé
4,2018-08-16T09:23:18+02:00,NaT,Paris Montparnasse,87391003.0,Sac à dos,"Bagagerie: sacs, valises, cartables",Objet trouvé
...,...,...,...,...,...,...,...
907271,2024-08-30T19:59:22+02:00,2024-09-04 12:34:26+02:00,Strasbourg,87212027.0,"Valise, sac sur roulettes","Bagagerie: sacs, valises, cartables",Objet trouvé
907272,2024-08-30T20:55:27+02:00,2024-09-05 12:49:08+02:00,Lille Europe,87223263.0,"Valise, sac sur roulettes","Bagagerie: sacs, valises, cartables",Objet trouvé
907273,2024-08-30T23:23:44+02:00,NaT,Nîmes,87775007.0,Sac à dos,"Bagagerie: sacs, valises, cartables",Objet trouvé
907274,2024-08-30T23:36:00+02:00,2024-08-31 12:43:30+02:00,Nice,87756056.0,"Ordinateur, ordinateur portable, notebook prot...","Appareils électroniques, informatiques, appare...",Objet trouvé


In [21]:
df.isna().sum()

Date                                 0
Date et heure de restitution    556378
Gare                              1079
Code UIC                          1079
Nature d'objets                      0
Type d'objets                        0
Type d'enregistrement                0
dtype: int64

In [22]:
(df.isna().sum()/len(df['Gare']))*100

Date                             0.000000
Date et heure de restitution    61.324007
Gare                             0.118927
Code UIC                         0.118927
Nature d'objets                  0.000000
Type d'objets                    0.000000
Type d'enregistrement            0.000000
dtype: float64

In [7]:
df[df['Gare'].isna()].isna().sum()

Date                               0
Date et heure de restitution     893
Gare                            1079
Code UIC                        1079
Nature d'objets                    0
Type d'objets                      0
Type d'enregistrement              0
dtype: int64

### Traitement des valeurs manquantes 

In [44]:
dictio = {'valeur' : np.nan}
#df['Gare'] = df['Gare'].replace(dictio=='Pas de Gare')

df['Gare'] = df['Gare'].fillna('Gare Inconnue')
df['Code UIC'] = df['Code UIC'].fillna('Code Inconnu')
#df['Date et heure de restitution'].fillna('Temporalité inconnue')
df = df.drop('Date et heure de restitution',axis=1)

## Valeurs incorrectes 

## Analyse graphique 

## Échantillonnage 

In [50]:
random.seed(345)
df_echan = df.sample(100000)

df_echan

,Date,Gare,Code UIC,Nature d'objets,Type d'objets,Type d'enregistrement
92995,2019-10-23T12:59:32+02:00,Paris Gare de Lyon,87686006.0,Lunettes,Optique,Objet trouvé
402903,2017-05-10T12:32:26+02:00,Nancy,87141002.0,"Téléphone portable protégé (étui, coque,…)","Appareils électroniques, informatiques, appare...",Objet trouvé
667313,2017-07-06T17:09:00+02:00,La Rochelle,87485003.0,Téléphone portable,"Appareils électroniques, informatiques, appare...",Objet trouvé
670923,2017-07-18T09:07:51+02:00,Le Havre,87413013.0,"Ordinateur, ordinateur portable, notebook prot...","Appareils électroniques, informatiques, appare...",Objet trouvé
268521,2022-10-11T08:30:21+02:00,Bergerac,87584268.0,Gants,"Vêtements, chaussures",Objet trouvé
...,...,...,...,...,...,...
268971,2022-10-12T17:48:12+02:00,Paris Gare de Lyon,87686006.0,"Diplôme, papiers personnels",Pièces d'identités et papiers personnels,Objet trouvé
730485,2016-06-08T14:05:54+02:00,Paris Gare du Nord,87271007.0,"Carte d'identité, passeport, permis de conduire",Pièces d'identités et papiers personnels,Objet trouvé
154616,2021-08-24T19:34:49+02:00,Poitiers,87575001.0,Autre pièce ou papier personnel,Pièces d'identités et papiers personnels,Objet trouvé
206938,2019-04-13T12:05:02+02:00,Strasbourg,87212027.0,"Bonnet, chapeau","Vêtements, chaussures",Objet trouvé
